https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html#scipy.integrate.odeint

In [ ]:
import numpy as np
import k3d
from ratelimiter import RateLimiter
from scipy.integrate import odeint

def pend(y, t, b, c):
    theta, omega = y
    dydt = [omega, -b*omega - c*np.sin(theta)]
    return dydt

b = 0.25
c = 5.0
y0 = [np.pi - 0.1, 0.0]
t = np.linspace(0, 25, 400)
sol = odeint(pend, y0, t, args=(b, c))

plot = k3d.plot(grid_auto_fit=False, camera_auto_fit=False,)
point = k3d.points([0, np.sin(sol[0,0]), -np.cos(sol[0,0])], point_size=0.1)
line = k3d.line([0, np.sin(sol[0,0]), 0])
plot += point + line
plot.display()

In [ ]:
rate_limiter = RateLimiter(max_calls=1, period=1/30)

for i in range(1, sol.shape[0]):
    with rate_limiter:
        point.positions = [0, np.sin(sol[i,0]), -np.cos(sol[i,0])]
        line.vertices = np.float32( np.concatenate([line.vertices, np.array([0, np.sin(sol[i,0]), 0])]) 
                                    - np.tile(np.array([0.01,0,0]), np.int(line.vertices.shape[0]/3 + 1)))

In [ ]:
line.color_map = k3d.basic_color_maps.Jet
line.color_range = 0, max(sol[:,1])
line.attribute = np.float32(abs(sol[:,1]))